# Advent of Code 2019 Day 3 - Intersections and Manhattan Distance

The problem for Day 3 provides a series of line segments and asks for the closest intersection to the starting point.  The full problem statement can be found on the [Advent of Code][1] website.  The key portions of the problem statement are below.

## Day 3a

> ... two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).
>
> The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. ...

# Solution - Day 3a

There are a few items to consider in the problem statement.  

1. We are not interested in situations where a single wire intersects with itself.
1. Intersections are between [line segments][2], not lines (which are a geometric construct).  In short, a line segment is a finite portion of a line with a start point, an end point, and a length.  The implication is that there are situations where two lines may intersect but line segments on those lines do not.
1. The wires are on a grid and we have to use [Manhattan distance][3] to calculate the distance from the central port. Manhattan distance or Taxicab geometry states that the distance between two points "is the sum of the absolute differences of their Cartesian coordinates".  For example, the distance from `(1,2)` to `(7,8)` is `12` (`|7-1|+|8-2|`).

[1] https://adventofcode.com/2019/day/3
[2] http://www.differencebetween.net/science/difference-between-line-and-line-segment/
[3] https://en.wikipedia.org/wiki/Taxicab_geometry
[4] https://en.wikipedia.org/wiki/Bentley%E2%80%93Ottmann_algorithm
[5] https://en.wikibooks.org/wiki/F_Sharp_Programming/Advanced_Data_Structures#Red_Black_Trees
[6] https://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf
[7] https://github.com/kazu-yamamoto/llrbtree/blob/master/Data/Set/LLRBTree.hs
[8] https://github.com/Skinney/core/blob/master/src/Dict.elm

In [5]:
/// implement:
/// https://github.com/pgkelley4/line-segments-intersect/blob/master/js/line-segments-intersect.js
/// https://stackoverflow.com/questions/563198/how-do-you-detect-where-two-line-segments-intersect
/// https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/
/// https://stackoverflow.com/questions/563198/how-do-you-detect-where-two-line-segments-intersect/565282#565282
/// https://bell0bytes.eu/intersection-of-line-segments/
/// https://en.wikipedia.org/wiki/Line_segment_intersection
/// https://stackoverflow.com/questions/385305/efficient-maths-algorithm-to-calculate-intersections
/// http://mathonline.wikidot.com/finding-points-of-intersection-of-two-lines
/// 
/// https://en.wikipedia.org/wiki/Taxicab_geometry
/// https://en.wikipedia.org/wiki/Bentley–Ottmann_algorithm

let x = 5

printfn "%d" x

5


## Strategies

There are two ways to solve the problem for 3a.

1. Create the line segments for each wire, and check which line segments intersect.  For this, we can use the [Bentley-Ottmann algorithm][4].
1. Find all the integer-coordinates covered by the line segments.  Check which coordinates are the same in both sets.

## Solution 3a.1 - Bentley-Ottmann algorithm

The [Bentley-Ottmann algorithm][4] is a "sweep line algorithm for listing all crossings in a set of line segments, i.e. it finds the intersection points (or, simply, intersections) of line segments".  The complexity of the algorithm is as follows, given `n` input line segments and `k` crossings:

- Time complexity: `O((n + k) log n)`
- Space complexity: `O(n)`

From a space perspective, the algorithm uses two data structures:

- The sweep line status tree (balanced binary search tree) that holds the line segments that cross the sweep line, ordered by the `y` coordinates of the intersection points.
    - This can be represented by a red-black tree or similar tree that has logarithmic time complexity (`O(log n)`) for inserts, deletes, and searches.
- The event queue (priority queue) that holds the sequence of potential future events (represented as points which are either line segment endpoints or cross points).  An event occurs when the sweep line sweeps over the point.  It is prioritized by the `x` coordinates of the points.
    - This can be represented by a binary heap or similar logarithmic-time priority queue.

The following is the logic and pseudo-code for the algorithm:

1. Initialize `Q` (priority queue) with the endpoints of all the input segments.
1. Initialize `T` (binary search tree) as an empty tree.
1. While `Q` is not empty,
    1. Get `p` based on minimum x-coordinate.
    1. If `p` = left endpoint of segment `s`,
        1. Insert `s` into `T`.
        1. Find segments `r` and `t`, which are above and below `s` in `T` (if they exist).
        1. If `r` crosses `t`,
            1. Remove that event from `Q`.
        1. If `s` crosses `r` or `t`,
            1. Add those crossing points to `Q`.
    1. If `p` = right endpoint of segment `s`,
        1. Remove `s` from `T`.
        1. Find segments `r` and `t`, which were above and below `s` in `T` prior to removal (if they exist).
        1. If `r` crosses `t`,
            1. Add that cross point to `Q`.
    1. If `p` = crossing point of segments `s` and `t` (`s` below `t` to the left of the crossing),
        1. Swap the position of `s` and `t` in `T`.
        1. Find segments `r` and `u`, which are above and below `s` and `t`, respectively.
        1. If `r` crosses `s`,
            1. Remove that event from `Q`.
        1. If `t` crosses `u`,
            1. Remove that event from `Q`.
        1. If `r` crosses `t`,
            1. Add that event to `Q`.
        1. If `s` crosses `u`,
            1. Add that event to `Q`.

The original algorithm makes certain assumptions which must be handled.  These include:

- No two endpoints or crossings should have the same `x` coordinate.
    - If two event points have the same `x` coordinate, prioritize the events using their `y` coordinates.
- No segment's endpoint should lie on another segment
    - If two segments share an endpoint or a segment contains the endpoint of another segments, count this as an intersection of two line segments.
- No three (or more) segments should intersect at a single point.
    - If multiple segments intersect at the same point, create and process only a single event point for that intersection.  This means:
        - Remove segments from the tree for which this point is the right endpoint.
        - Insert segments into the tree for which this point is the left endpoint.
        - Reverse the order of the remaining segments containing this point.

For this problem, we are only dealing with perfectly vertical and horizontal lines, and integer coordinates.  As such, there should be no issues related to numerical precision.

The solution starts with the [data structures][5].  However, for this problem, I chose a [left-leaning red-black tree][6] based on [Haskell][7] and [Elm][8] implementations.  The F# repository linked from the Wikipedia page appears to have been deleted.  Left-leaning red-black trees are considered slightly easier to implement as they reduce the number of variations that need to be accounted for in different methods.

In [3]:
// Possible colors of nodes.
type Color = R | B  

type 'a tree =
  | E
  | T of Color *  * 'a * 'a tree * 'a tree

module Tree =
  let hd = function
    | E -> failwith "Empty tree"
    | T(_, _, x, _) -> x

  let left = function
    | E -> failwith "Empty tree"
    | T(_, l, _, _) -> l

  let right = function
    | E -> failwith "Empty tree"
    | T(_, _, _, r) -> r

  let rec exists item = function
    | E -> false
    | T(_, l, x, r) ->
      if item = x then true
      elif item < x then exists item l
      else exists item r

  let balance = function
    | B, T (R, T (R,a,x,b), y, c), z, d
    | B, T (R, a, x, T (R,b,y,c)), z, d
    | B, a, x, T (R, T (R,b,y,c), z, d)
    | B, a, x, T (R, b, y, T (R,c,z,d)) -> 
      T (R, T (B,a,x,b), y, T (B,c,z,d))
    | col, a, x, b -> T (col, a, x, b) 

  let insert elt t =
    let rec ins = function
      | E -> T(R, E, elt, E)
      | T(c, a, y, b) as node ->
        if item = y then node
        elif item < y then balance(c, ins a, y, b)
        else balance(c, a, y, ins b)
      
    match ins t with
      | E -> failwith "Should never return empty from an insert"
      | T(_, l, x, r) -> T(B, l, x, r)
          
  let delete elt t =
    

Stopped due to error


Unhandled exception: input.fsx (27,1)-(27,7) parse error Incomplete structured construct at or before this point in binding
input.fsx (5,1)-(5,10) typecheck error This value is not a function and cannot be applied.
input.fsx (23,38)-(23,39) typecheck error Type mismatch. Expecting a
    ''a tree'    
but given a
    ''a'    
The types ''a' and ''a tree' cannot be unified.
input.fsx (26,17)-(26,25) typecheck warning Incomplete pattern matches on this expression.


## Solution 3a.2 - Check discrete coordinates

For this solution, we must construct a set of all the discrete (integer) coordinates that a wire goes through.

In [ ]:


let moveToCoords startPoint direction distance =
  x

let 3a_2 wire1 wire2